In [2]:
import os
from IPython.display import Audio

In [8]:
applause_dirs = ['/data/jrgillick/applause-training-data/applause_pt1/', '/data/jrgillick/applause-training-data/applause_pt2/']
non_applause_dirs = ['/data/jrgillick/applause-training-data/non_applause_pt1/','/data/jrgillick/applause-training-data/non_applause_pt2/']

In [68]:
applause_files = [[f for f in os.listdir(d)] for d in applause_dirs]
applause_paths = [applause_dirs[0] + f for f in applause_files[0]] + [applause_dirs[1] + f for f in applause_files[1]]
applause_files = applause_files[0] + applause_files[1]

non_applause_files = [[f for f in os.listdir(d)] for d in non_applause_dirs]
non_applause_paths = [non_applause_dirs[0] + f for f in non_applause_files[0]] + [non_applause_dirs[1] + f for f in non_applause_files[1]]
non_applause_files = non_applause_files[0] + non_applause_files[1]

In [116]:
import keras
import numpy as np
import librosa
from sklearn.utils import shuffle
import pandas as pd

In [70]:
applause_labels = pd.read_csv('/data/jrgillick/applause-training-data/PennSound_applause_labels.csv',header=None)
non_applause_labels = pd.read_csv('/data/jrgillick/applause-training-data/PennSound_non_applause_labels.csv',header=None)

In [71]:
applause_labels.columns = ['File','Start','End']
non_applause_labels.columns = ['File','Start','End']

<h2> Extract Features </h2>

In [162]:
def extract_features(f):
    try:
        y, sr = librosa.load(f)
        mfcc = librosa.feature.mfcc(y,n_mfcc=13)
        delta = librosa.feature.delta(mfcc)
        return np.vstack([mfcc,delta])
    except:
        print "%s failed" % (f)

def extract_features_from_files(files):
    return [extract_features(f) for f in files]

def get_features_and_labels(S,window_size):
    features = []
    for i in range(window_size,S.shape[1]-window_size):
        feature = S[:,i-window_size:i+window_size]
        features.append(feature.reshape((-1)))
    return features

In [108]:
applause_features = extract_features_from_files(applause_paths)
applause_features = [f for f in applause_features if f is not None]
non_applause_features = extract_features_from_files(non_applause_paths)

/data/jrgillick/applause-training-data/applause_pt1/Yankelevich_Ostashevsky_04_Eugene_KWH-UPenn_10-11-2012__1469.0_1472.0.wav failed


In [166]:
test_set_index = int(len(applause_features) * 0.2)
test_set_index

55

In [176]:
test_applause_feature_list = [np.array(get_features_and_labels(f,5)) for f in applause_features[0:test_set_index]]
test_applause_features = np.vstack(test_applause_feature_list)
test_applause_labels = np.ones(len(test_applause_features))

test_non_applause_feature_list = [np.array(get_features_and_labels(f,5)) for f in non_applause_features[0:test_set_index]]
test_non_applause_features = np.vstack(test_non_applause_feature_list)
test_non_applause_labels = np.ones(len(test_non_applause_features))

In [180]:
train_applause_feature_list = [np.array(get_features_and_labels(f,5)) for f in applause_features[test_set_index:]]
train_applause_features = np.vstack(train_applause_feature_list)
train_applause_labels = np.ones(len(train_applause_features))

train_non_applause_feature_list = [np.array(get_features_and_labels(f,5)) for f in non_applause_features[test_set_index:]]
train_non_applause_features = np.vstack(train_non_applause_feature_list)
train_non_applause_labels = np.ones(len(train_non_applause_features))

In [196]:
X_train = np.vstack([train_applause_features,train_non_applause_features])
y_train = np.concatenate([train_applause_labels,train_non_applause_labels])

In [197]:
X_test = np.vstack([test_applause_features,test_non_applause_features])
y_test = np.concatenate([test_applause_labels,test_non_applause_labels])

In [280]:
X_train, y_train = shuffle(X_train,y_train)
X_test, y_test = shuffle(X_test, y_test)

In [281]:
for i in xrange(X_train.shape[1]):
    mean = np.mean(X_train[:,i])
    X_train[:,i] -= mean
    X_test[:,i] -=mean
    std = np.std(X_train[:,i])
    X_train[:,i] /= std
    X_test[:,i] /= std

<h2> Run Model </h2>

In [282]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Dropout
import keras.optimizers
from keras.models import load_model
import keras.regularizers
from keras.regularizers import l2, l1
from keras.layers import LSTM
from keras.layers import regularizers
from keras.layers import Bidirectional

In [283]:
def initialize_ff_model():
    model = Sequential()
    model.add(Dense(100, use_bias=True, kernel_regularizer=regularizers.l2(0.5), input_dim=260))
    model.add(Dropout(0.5))
    model.add(Activation("relu"))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [284]:
model = initialize_ff_model()

In [293]:
model.fit(X_train,y_train,epochs=1,batch_size=256,shuffle=True)

Epoch 1/1
89542/89542 [==============================] - 1s - loss: 0.0014 - acc: 1.0000     


In [294]:
model.evaluate(X_test,y_test,batch_size=256)

11008/13199 [========================>.....] - ETA: 0s

[0.00068322052848112215, 1.0]